In [1]:
import argparse
import networkx as nx
import numpy as np
import scipy as sp
import tensorflow as tf
import my
import objectives
import partition as pttn

/home/alex/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [3]:
args = argparse.Namespace()
args.device = '/cpu:0'
args.graph = 'soc-Epinions1'
args.lambda_node = 1.0
args.lambda_edge = 1.0
args.lambda_entropy = 1.0
args.partitioner = 'greedy'
# args.partitioner = 'random'
args.n_machines = 10

In [4]:
g = my.read_edgelist('data/' + args.graph, fill=True)
adj = nx.adj_matrix(g)
adj = sp.sparse.triu(adj)

In [5]:
rng = np.random.RandomState(42)
partitioner = {
    'greedy' : pttn.GreedyVertexCut,
    'random' : pttn.RandomVertexCut,
}[args.partitioner](adj, args.n_machines, rng)

In [6]:
partitioner.partition()
z = tf.one_hot(tf.constant(partitioner.eassign), args.n_machines)

In [7]:
with tf.device(args.device):
    objective = objectives.Objective(adj, args.n_machines, args.lambda_node, args.lambda_edge, args.lambda_entropy)

In [8]:
with tf.device(args.device):
    rslt, r, b_node, b_edge, entropy = objective(z, training=True, probability=True)
    print('%f %f %f %f %f' % (rslt, r, b_node, b_edge, entropy))
    r, b_node, b_edge, entropy = objective(z, probability=True)
    print('%f %f %f %f' % (r, b_node, b_edge, entropy))

-0.000000 1.854667 -2.295276 -2.309941 -0.000000
1.854667 -2.295276 -2.309941 -0.000000
